### Universidad del Valle de Guatemala
#### Brandon Ronaldo Sicay Cumes - 21757
##### Laboratorio 1 - Parte A

In [5]:
# CHAT GPT-03-MINI-HIGH: https://chatgpt.com/c/679043bd-6990-800f-8b61-e48f1ddf164c

import unicodedata
import matplotlib.pyplot as plt
import itertools


In [6]:
from programs.caesar import Ceasar
from programs.afin import Afin
from programs.vigenere import Vigenere
from programs.frequency import *

In [4]:
sp_frequencies = {
  "a": 0.11525,
  "b": 0.02215,
  "c": 0.04019,
  "d": 0.05010,
  "e": 0.12181,
  "f": 0.00692,
  "g": 0.01768,
  "h": 0.00703,
  "i": 0.06247,
  "j": 0.00493,
  "k": 0.00011,
  "l": 0.04967,
  "m": 0.03157,
  "n": 0.06712,
  "o": 0.08683,
  "p": 0.02510,
  "q": 0.00877,
  "r": 0.06871,
  "s": 0.07977,
  "t": 0.04632,
  "u": 0.02927,
  "v": 0.01138,
  "w": 0.00017,
  "x": 0.00215,
  "y": 0.01008,
  "z": 0.00467,
  "á": 0.00502,
  "é": 0.00433,
  "í": 0.00725,
  "ñ": 0.00311,
  "ó": 0.00827,
  "ú": 0.00168,
  "ü": 0.00012
}

In [11]:
def distribution_distance(dist_test, dist_ref):
    """
    Retorna una 'distancia' entre dos distribuciones.
    A menor valor, mayor similitud.
    """
    distance = 0.0
    # Iterar sobre las letras que tengamos en la referencia
    for letter, ref_val in dist_ref.items():
        test_val = dist_test.get(letter, 0.0)
        distance += (test_val - ref_val)**2
    return distance


### Brute Force Caesar

In [21]:
# def brute_force_cesar(distribuciones, probabilidad_real, mensaje_cifrado):
#     letras = 'abcdefghijklmnñopqrstuvwxyz'
#     letra_mas_frecuente_dist = max(distribuciones, key=distribuciones.get)
#     letra_mas_frecuente_real = max(probabilidad_real, key=probabilidad_real.get)

#     desplazamiento_inicial = (letras.index(letra_mas_frecuente_real) - letras.index(letra_mas_frecuente_dist)) % len(letras)
#     print(f"Desplazamiento inicial: {desplazamiento_inicial}")
#     indice_inicial = letras[letras.index(letra_mas_frecuente_real) - desplazamiento_inicial]

#     # Probando desplazamientos a partir del desplazamiento inicial
#     with open('ceasar.txt', 'w', encoding='utf-8') as file:
#         for i in range(31):
#             desplazamiento_actual = (desplazamiento_inicial + i) % len(letras)
#             cesar = Ceasar(desplazamiento_actual)
#             mensaje_descifrado = cesar.c_decrypt(mensaje_cifrado)
#             file.write(f"Desplazamiento: {desplazamiento_actual} - Mensaje: {mensaje_descifrado}\n")


def brute_force_cesar(mensaje_cifrado, sp_frequencies):
    """
    Prueba todas las llaves (0..30) para el cifrado César con el alfabeto 
    'abcdefghijklmnñopqrstuvwxyz'. 
    Calcula la distancia de cada desencriptado respecto a las frecuencias 
    del español y retorna los resultados ordenados.
    """
    alfabeto = 'abcdefghijklmnñopqrstuvwxyz'
    resultados = []

    for key in range(31):  # 0..30
        # Instancia de Ceasar con la clave 'key'
        cesar = Ceasar(key)
        mensaje_descifrado = cesar.c_decrypt(mensaje_cifrado)
        
        # Calculamos la distribución de este texto desencriptado
        dist_descifrado = calcular_distribucion(mensaje_descifrado)
        
        # Calculamos la distancia
        dist_medida = distribution_distance(dist_descifrado, sp_frequencies)
        
        resultados.append((key, dist_medida, mensaje_descifrado))
    
    # Ordenar por la distancia (menor distancia = mejor similitud)
    resultados.sort(key=lambda x: x[1])
    
    return resultados

In [23]:
# ceasar_encrypted = ''
# with open('./cipher/ceasar.txt', 'r', encoding='utf-8') as archivo:
#             ceasar_encrypted = archivo.read()

# distribuciones = calcular_distribucion(ceasar_encrypted)
# brute_force_cesar(distribuciones, sp_frequencies, ceasar_encrypted)


with open('cipher/ceasar.txt', 'r', encoding='utf-8') as archivo:
    ceasar_encrypted = archivo.read()
    
# Ejecutar la fuerza bruta
resultados_ordenados = brute_force_cesar(ceasar_encrypted, sp_frequencies)

# Guardar (o imprimir) los resultados del mejor al peor
with open('ceasar_resultados.txt', 'w', encoding='utf-8') as file:
    for key, dist_val, msg in resultados_ordenados:
        linea = f"Desplazamiento: {key} | Distancia: {dist_val:.6f} | Mensaje: {msg}\n"
        file.write(linea)

# O bien, imprimir en pantalla solo las mejores 5 claves:
print("Las 5 claves más probables:")
for key, dist_val, msg in resultados_ordenados[:5]:
    print(f"Key: {key}, Dist: {dist_val:.6f}, Decifrado: {msg[:60]}...")    

Las 5 claves más probables:
Key: 23, Dist: 767.065842, Decifrado: nnuestrolaberintodigitalenconstanteevolucionlaagilidadcripto...
Key: 0, Dist: 772.114598, Decifrado: jjqaopñlhwxañejplzecepwhajyljopwjpaarlhqyeljhwwcehezwzyñempl...
Key: 27, Dist: 772.114598, Decifrado: jjqaopñlhwxañejplzecepwhajyljopwjpaarlhqyeljhwwcehezwzyñempl...
Key: 19, Dist: 772.265850, Decifrado: qqyiwxvsoefivmqxshmkmxeoiqgsqwxeqxiizsoygmsqoeekmomhehgvmtxs...
Key: 8, Dist: 772.375168, Decifrado: bbjshigdzopsgwbidrwuwiozsbqdbhiobisskdzjqwdbzoouwzwrorqgweid...


### Afín

Número a = Rango del 1-16
Número b = Rango del 1-16
Texto contiene: extraer de los ataques

In [24]:
def brute_force_afin(mensaje_cifrado, sp_frequencies):
    """
    Recorre todas las posibles claves a=1..16, b=1..16.
    Desencripta, calcula la distribución y la compara con 
    las frecuencias del español (sp_frequencies).
    Retorna una lista de tuplas (a, b, distancia, mensaje_descifrado) ordenada.
    """
    resultados = []
    
    # Recorremos todos los a=1..16, b=1..16
    for a in range(1,17):
        for b in range(1,17):
            try:
                cipher_afin = Afin(a, b)
                mensaje_descifrado = cipher_afin.decrypt(mensaje_cifrado)
                
                # Calcular la distribución y la distancia
                dist_descifrado = calcular_distribucion(mensaje_descifrado)
                dist_val = distribution_distance(dist_descifrado, sp_frequencies)
                
                resultados.append((a, b, dist_val, mensaje_descifrado))
            
            except:
                # Si gcd(a, 27)!=1 o surge algún otro error, lo ignoramos
                pass
    
    # Ordenar por la distancia (menor = mejor)
    resultados.sort(key=lambda x: x[2])
    return resultados


In [25]:
with open('./cipher/afines.txt', 'r', encoding='utf-8') as archivo:
    afin_encrypted = archivo.read()

# Llamar a la fuerza bruta
resultados_afin = brute_force_afin(afin_encrypted, sp_frequencies)

# Guardar todos en un archivo
with open('afin_resultados.txt', 'w', encoding='utf-8') as f:
    for (a, b, dist_val, msg_descifrado) in resultados_afin:
        linea = (f"a={a}, b={b}, dist={dist_val:.6f}, "
                    f"descifrado={msg_descifrado}\n")
        f.write(linea)

# Mostrar los 5 mejores en consola
print("Las 5 claves más probables para el cifrado afín:")
for (a, b, dist_val, msg_descifrado) in resultados_afin[:5]:
    print(f"[a={a}, b={b}] Dist={dist_val:.6f}, Decifrado => {msg_descifrado[:60]}...")

Las 5 claves más probables para el cifrado afín:
[a=5, b=15] Dist=731.463758, Decifrado => nejemplodelanecesidaddecriptoagilidadsepuedeextraerdelosataq...
[a=14, b=15] Dist=734.965461, Decifrado => vnjnmycodncavntnbzdaddntrzyloagzczdadbnyundnnxlranrdncobalai...
[a=4, b=8] Dist=735.616265, Decifrado => ranakochsacvralamfsvssalefothvpfcfsvsmaobasaaytevaesachmvtvw...
[a=11, b=15] Dist=735.631037, Decifrado => bsjsdvfxusfabswspluauuswrlvñxaolfluaupsvmsussgñrasrusfxpañat...
[a=13, b=3] Dist=735.796259, Decifrado => iquqrfboaqbdiqkqceadaaqkmefsodxebeadacqfjqaqqgsmdqmaqbocdsdv...


### Veginere

In [12]:
import itertools

def brute_force_vigenere(mensaje_cifrado, sp_frequencies, output_file='vigenere_resultados.txt'):
    """
    Busca claves de longitud 2..6 que INICIEN con 'pa'.
    Para cada clave, desencripta, calcula la distancia de frecuencias

    """
    alfabeto = 'abcdefghijklmnñopqrstuvwxyz'
    resultados = []

    # Recorremos la longitud de clave de 2 a 6
    for key_length in range(2, 7):
        # Caso especial: si key_length = 2, la clave es 'pa' sin más
        if key_length == 2:
            clave = 'pa'
            vigenere_cipher = Vigenere(clave)
            descifrado = vigenere_cipher.decrypt(mensaje_cifrado)

            dist_test = calcular_distribucion(descifrado)
            dist_val = distribution_distance(dist_test, sp_frequencies)
            resultados.append((clave, dist_val, descifrado))
        
        else:
            # Para key_length > 2, generamos las letras faltantes
            # (key_length - 2) posiciones por llenar
            for comb in itertools.product(alfabeto, repeat=key_length - 2):
                # 'pa' + el resto
                clave = 'pa' + ''.join(comb)
                vigenere_cipher = Vigenere(clave)
                descifrado = vigenere_cipher.decrypt(mensaje_cifrado)

                dist_test = calcular_distribucion(descifrado)
                dist_val = distribution_distance(dist_test, sp_frequencies)

                resultados.append((clave, dist_val, descifrado))
    
    # Ordenar por distancia (menor = mejor)
    resultados.sort(key=lambda x: x[1], reverse=True)

    
    # Guardar todos en un archivo
    with open(output_file, 'w', encoding='utf-8') as f:
        for (clave, dist_val, msg) in resultados:
            f.write(f"Clave={clave}, Distancia={dist_val:.6f}, Descifrado={msg}\n")
    
    return resultados


In [13]:
with open('./cipher/vigenere.txt', 'r', encoding='utf-8') as archivo:
    vigenere_encrypted = archivo.read()

# Ejecutar fuerza bruta
resultados_ordenados = brute_force_vigenere(vigenere_encrypted, sp_frequencies)

print("Las 5 claves más probables para Vigenère con prefijo 'pa':")
for (clave, dist_val, descifrado) in resultados_ordenados[:5]:
    print(f"Clave={clave}, Dist={dist_val:.6f}, Mensaje => {descifrado[:60]}...")

Las 5 claves más probables para Vigenère con prefijo 'pa':
Clave=payaso, Dist=737.419479, Mensaje => aunqueelerrorheartbleedsehasolucionadosiemprehayunanuevaamen...
Clave=payaho, Dist=683.641114, Mensaje => aunqgeelerdorheadtbleeosehasalucioyadosipmprehmyunangevaampn...
Clave=paypso, Dist=680.403447, Mensaje => aunbueelecrorhelrtbleodsehadoluciznadossemprerayunaxuevaawen...
Clave=payaio, Dist=679.055436, Mensaje => aunqfeelercorheactbleeñsehaszlucioxadosiomprehlyunanfevaamon...
Clave=payado, Dist=676.891776, Mensaje => aunqkeelerhorheahtbleessehaseluciocadositmprehpyunankevaamtn...
